<a href="https://colab.research.google.com/github/KanimozhiShadows/Python_Repo/blob/main/File%20monitoring%20using%20FastAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install watchdog
!pip install apscheduler
!pip install mysql-connector-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 51.6 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from apscheduler.schedulers.background import BackgroundScheduler
import mysql.connector
import time
import os

In [4]:
# ---------- CONFIGURATION ----------
WATCH_DIRECTORY = "D:\Kani\Projects\Python based\Py_File_Db\filelist"  # <-- Change this to the folder you want to monitor
DB_CONFIG = {
    "host": "localhost",
    "user": "kani",
    "password": "kani2310",
    "database": "storage"
}


In [5]:
# ---------- DATABASE CONNECTION ----------

def insert_file_to_db(filename, filepath):
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        cursor = connection.cursor()
        sql = "INSERT INTO files (filename, filepath) VALUES (%s, %s)"
        values = (filename, filepath)
        cursor.execute(sql, values)
        connection.commit()
        cursor.close()
        connection.close()
        print(f"Inserted {filename} into database.")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

In [6]:
# ---------- FILE EVENT HANDLER ----------

class FileEventHandler(FileSystemEventHandler):
    def on_created(self, event):
        if not event.is_directory:
            filename = os.path.basename(event.src_path)
            filepath = event.src_path
            print(f"Detected new file: {filename}")
            insert_file_to_db(filename, filepath)

In [7]:
# ---------- SCHEDULER TASK ----------

def start_watchdog():
    event_handler = FileEventHandler()
    observer = Observer()
    observer.schedule(event_handler, path=WATCH_DIRECTORY, recursive=False)
    observer.start()
    print(f"Started watching {WATCH_DIRECTORY}")
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

In [9]:
# ---------- FASTAPI SETUP ----------

from contextlib import asynccontextmanager

@app.get("/")
def read_root():
    return {"message": "File listener is running!"}

@asynccontextmanager
async def lifespan(app: FastAPI):
    # This block runs on startup
    scheduler = BackgroundScheduler()
    scheduler.add_job(start_watchdog)
    scheduler.start()
    print("Scheduler started!")

    yield

    # This block runs on shutdown
    scheduler.shutdown()
    print("Scheduler shutdown!")

# Attach lifespan to app
app = FastAPI(lifespan=lifespan)


In [ ]:
#to run this: save it in main.py
# then rum this in terminal.
#uvicorn main:app --reload
